In [1]:
%load_ext autoreload
%autoreload 2

# Play around with openfold losses

In [50]:
from openfold.utils.loss import lddt_loss, AlphaFoldLoss

In [52]:
import openfold

In [53]:
openfold.__file__

'/homefs/home/lux70/openfold/openfold/__init__.py'

In [56]:
out = recons_struct[1]
print(out.keys())

dict_keys(['frames', 'sidechain_frames', 'unnormalized_angles', 'angles', 'positions', 'states', 's_initial', 's_after_layernorm', 'z_after_layernorm', 'single', 'sm_s', 'sm_z', 's_s', 's_z', 'distogram_logits', 'lm_logits', 'aatype', 'atom14_atom_exists', 'residx_atom14_to_atom37', 'residx_atom37_to_atom14', 'atom37_atom_exists', 'residue_index', 'lddt_head', 'plddt', 'ptm_logits', 'ptm', 'aligned_confidence_probs', 'predicted_aligned_error', 'max_predicted_aligned_error'])


In [64]:
from openfold.config import model_config
config = model_config(name="initial_training", train=True)

In [65]:
config.loss

distogram:
  eps: &id001 !!python/object:ml_collections.config_dict.config_dict.FieldReference
    _field_type: !!python/name:builtins.float ''
    _ops: []
    _required: false
    _value: 1.0e-08
  max_bin: 21.6875
  min_bin: 2.3125
  no_bins: 64
  weight: 0.3
eps: *id001
experimentally_resolved:
  eps: *id001
  max_resolution: 3.0
  min_resolution: 0.1
  weight: 0.0
fape:
  backbone:
    clamp_distance: 10.0
    loss_unit_distance: 10.0
    weight: 0.5
  eps: 0.0001
  sidechain:
    clamp_distance: 10.0
    length_scale: 10.0
    weight: 0.5
  weight: 1.0
masked_msa:
  eps: *id001
  weight: 2.0
plddt_loss:
  cutoff: 15.0
  eps: *id001
  max_resolution: 3.0
  min_resolution: 0.1
  no_bins: 50
  weight: 0.01
supervised_chi:
  angle_norm_weight: 0.01
  chi_weight: 0.5
  eps: *id001
  weight: 1.0
tm:
  enabled: !!python/object:ml_collections.config_dict.config_dict.FieldReference
    _field_type: !!python/name:builtins.bool ''
    _ops: []
    _required: false
    _value: false
  eps: *

In [67]:
batch = batch[-1]

In [69]:
from openfold.utils.loss import distogram_loss
distogram_loss(out['distogram_logits'], **{**batch, **config.loss.distogram})

TypeError: distogram_loss() missing 2 required positional arguments: 'pseudo_beta' and 'pseudo_beta_mask'

In [72]:
from openfold.utils.loss import experimentally_resolved_loss
experimentally_resolved_loss(
    logits=out['experimentally_resolved_logits'],
    **{**batch, **config.loss.experimentally_resolved}
)

KeyError: 'experimentally_resolved_logits'

In [80]:
# from openfold.utils.loss import fape_loss
# wrapper_out = {"sm": out}
# fape_loss(wrapper_out, batch, config.loss.fape)

In [89]:
# from openfold.utils.loss import lddt_loss
# lddt_loss(
#     logits=out['plddt'], #TODO: these are not logits
#     all_atom_pred_pos=out['positions'],  #TODO: confim if these are the same
#     **{**batch, **config.loss.plddt_loss}
# )
# out.keys()

In [93]:
from openfold.utils.loss import supervised_chi_loss
supervised_chi_loss(
    out["angles"], out["unnormalized_angles"], **{**batch, **config.loss.supervised_chi}
)

TypeError: supervised_chi_loss() missing 3 required positional arguments: 'seq_mask', 'chi_mask', and 'chi_angles_sin_cos'

In [94]:
from openfold.utils.loss import violation_loss
violation_loss(
    out['violation'], **{**batch, **config.loss.violation}
)

KeyError: 'violation'

In [95]:
from openfold.utils.loss import tm_loss
tm_loss(
    logits=out['ptm_logits'], 
    **{**batch, **out, **config.loss.tm}
)

TypeError: tm_loss() missing 1 required positional argument: 'final_affine_tensor'

In [99]:
from openfold.utils.loss import chain_center_of_mass_loss
chain_center_of_mass_loss(
    all_atom_pred_pos=out['positions'],
    **{**batch, **config.loss.chain_center_of_mass}
)

ImportError: cannot import name 'chain_center_of_mass_loss' from 'openfold.utils.loss' (/homefs/home/lux70/openfold/openfold/utils/loss.py)